In [191]:
# -*- coding: utf-8 -*-
"""bayes_ejemplo.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/17bjDLrXxAWUwkh2ZunMOg9rwAU8D2EeO
"""

from functools import reduce

import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hipotesis = None


    def __init__(self,filename=None, class_attr=None ):
        self.data = pd.read_csv(filename, sep=',', header =(0))
        self.class_attr = class_attr

    '''
        probabilidad(clase) =   frecuencia de aparcicin or columna/ conteo clase
    '''
    def calcular_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data =  list(self.data[self.class_attr])
        for i in class_values:
            self.priori[i]  = class_data.count(i)/float(len(class_data))
        print ("Valores a priori : ", self.priori)

    '''
        Calculo de las probabilidades individuales
        P(outcome|evidencia) =   P(verosimilitud de la evidencia) x Prior prob de outcome/P(Evidencia)

    '''
    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total =1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total+=1
        return total/float(class_data.count(class_value))

    '''
        Aquí se calcula la Probabilidad de Evidencia y se multiplica todas las probabilidades individuales a priori
        (Outcome|Multiple Evidence) = P(Evidence1|Outcome) x P(Evidence2|outcome) x ... x P(EvidenceN|outcome) x P(Outcome)
        escalado por P(Multiple Evidence)
    '''
    def calcular_probabilidad_condicional(self, hipotesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hipotesis:
                self.cp[i].update({ hipotesis[j]: self.get_cp(j, hipotesis[j], i)})
        print ("\nProbabilidades condicionales calculadas: \n")
        pprint.pprint(self.cp)

    def clasificar(self):
        print ("Resultado: ")
        for i in self.cp:
            print (i, " ==> ", reduce(lambda x, y: x*y, self.cp[i].values())*self.priori[i])

if __name__ == "__main__":
    c = Classifier(filename="new_dataset.csv", class_attr="Play" )
    c.calcular_priori()
    c.hipotesis = {"Outlook":'Overcast',  "Temp":"Mild"}

    c.calcular_probabilidad_condicional(c.hipotesis)
    c.clasificar()

Valores a priori :  {'yes': 0.6428571428571429, 'no': 0.35714285714285715}

Probabilidades condicionales calculadas: 

{'no': {'Mild': 0.6, 'Overcast': 0.2},
 'yes': {'Mild': 0.5555555555555556, 'Overcast': 0.5555555555555556}}
Resultado: 
yes  ==>  0.19841269841269846
no  ==>  0.04285714285714286


In [192]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
file_path = 'loan.csv'  # Reemplaza con la ruta correcta de tu archivo CSV
df = pd.read_csv(file_path)
df.head()
print(df.columns)

df

Index(['loan_id', ' no_of_dependents', ' education', ' self_employed',
       ' income_annum', ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4264,4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4265,4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4266,4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected
4267,4268,1,Not Graduate,No,4100000,12800000,8,780,8200000,700000,14100000,5800000,Approved


In [193]:
df = df[[" education", " self_employed"," luxury_assets_value", " loan_status"]]

In [194]:
df[' luxury_assets_value'] = pd.qcut(df[' luxury_assets_value'], q=[0, 0.25, 0.5, 0.75, 1], labels=['Bajo', 'Medio Bajo', 'Medio Alto', 'Alto'])
df

C:\Users\Daphne\AppData\Local\Temp\ipykernel_38468\1543820807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[' luxury_assets_value'] = pd.qcut(df[' luxury_assets_value'], q=[0, 0.25, 0.5, 0.75, 1], labels=['Bajo', 'Medio Bajo', 'Medio Alto', 'Alto'])


,education,self_employed,luxury_assets_value,loan_status
0,Graduate,No,Alto,Approved
1,Not Graduate,Yes,Medio Bajo,Rejected
2,Graduate,No,Alto,Rejected
3,Graduate,No,Alto,Rejected
4,Not Graduate,Yes,Alto,Rejected
...,...,...,...,...
4264,Graduate,Yes,Bajo,Rejected
4265,Not Graduate,Yes,Medio Bajo,Approved
4266,Not Graduate,No,Medio Alto,Rejected
4267,Not Graduate,No,Medio Bajo,Approved


In [219]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Preprocesar los datos para usarlos con el algoritmo Apriori
te = TransactionEncoder()
te_ary = te.fit(df.values).transform(df.values)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Calcular itemsets frecuentes usando Apriori
frequent_itemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)

# Calcular reglas de asociación
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Imprimir resultados
print("Itemsets Frecuentes:")
print(frequent_itemsets)

print("\nReglas de Asociación:")
rules

Itemsets Frecuentes:
     support                          itemsets
0   0.622160                       ( Approved)
1   0.502225                       ( Graduate)
2   0.496369                             ( No)
3   0.497775                   ( Not Graduate)
4   0.377840                       ( Rejected)
5   0.503631                            ( Yes)
6   0.247599                            (Alto)
7   0.252752                            (Bajo)
8   0.250410                      (Medio Alto)
9   0.249239                      (Medio Bajo)
10  0.313657            ( Graduate,  Approved)
11  0.308737                  ( No,  Approved)
12  0.308503        ( Approved,  Not Graduate)
13  0.313422                 ( Approved,  Yes)
14  0.152963                 ( Approved, Alto)
15  0.164441                 ( Approved, Bajo)
16  0.153432           ( Approved, Medio Alto)
17  0.151323           (Medio Bajo,  Approved)
18  0.255095                  ( Graduate,  No)
19  0.188569            ( Rejected,  Gr

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,( Graduate),( Approved),0.502225,0.62216,0.313657,0.624534,1.003815,0.001192,1.006322,0.007636
1,( No),( Approved),0.496369,0.62216,0.308737,0.621992,0.999730,-0.000084,0.999555,-0.000537
2,( Not Graduate),( Approved),0.497775,0.62216,0.308503,0.619765,0.996150,-0.001192,0.993701,-0.007636
3,( Yes),( Approved),0.503631,0.62216,0.313422,0.622326,1.000267,0.000084,1.000439,0.000537
4,(Alto),( Approved),0.247599,0.62216,0.152963,0.617786,0.992970,-0.001083,0.988557,-0.009321
5,(Bajo),( Approved),0.252752,0.62216,0.164441,0.650602,1.045716,0.007189,1.081405,0.058505
6,(Medio Alto),( Approved),0.250410,0.62216,0.153432,0.612722,0.984831,-0.002363,0.975631,-0.020134
7,(Medio Bajo),( Approved),0.249239,0.62216,0.151323,0.607143,0.975863,-0.003743,0.961775,-0.031894
8,"( Graduate, No)",( Approved),0.255095,0.62216,0.159522,0.625344,1.005119,0.000812,1.008500,0.006837
9,"( Graduate, Yes)",( Approved),0.247130,0.62216,0.154134,0.623697,1.002470,0.000380,1.004084,0.003273


In [196]:
'''columns_to_convert = [" loan_status"]
df[columns_to_convert] = df[columns_to_convert].replace({' Approved': 1, ' Rejected': 0})'''
df

,education,self_employed,luxury_assets_value,loan_status
0,Graduate,No,Alto,Approved
1,Not Graduate,Yes,Medio Bajo,Rejected
2,Graduate,No,Alto,Rejected
3,Graduate,No,Alto,Rejected
4,Not Graduate,Yes,Alto,Rejected
...,...,...,...,...
4264,Graduate,Yes,Bajo,Rejected
4265,Not Graduate,Yes,Medio Bajo,Approved
4266,Not Graduate,No,Medio Alto,Rejected
4267,Not Graduate,No,Medio Bajo,Approved


In [213]:
prob_marginal_variable1 = df[" education"].value_counts(normalize=True)
prob_marginal_variable2 = df[' self_employed'].value_counts(normalize=True)
prob_marginal_variable3 = df[' luxury_assets_value'].value_counts(normalize=True)
prob_marginal_variable4 = df[' loan_status'].value_counts(normalize=True)

# Imprimir probabilidades marginales
print("Probabilidades Marginales de education:")
print(prob_marginal_variable1)

print("\nProbabilidades Marginales de self_employed:")
print(prob_marginal_variable2)

print("\nProbabilidades Marginales de luxury_assets_value:")
print(prob_marginal_variable3)

print("\nProbabilidades Marginales de loan_status:")
print(prob_marginal_variable4)

Probabilidades Marginales de education:
 education
 Graduate        0.502225
 Not Graduate    0.497775
Name: proportion, dtype: float64

Probabilidades Marginales de self_employed:
 self_employed
 Yes    0.503631
 No     0.496369
Name: proportion, dtype: float64

Probabilidades Marginales de luxury_assets_value:
 luxury_assets_value
Bajo          0.252752
Medio Alto    0.250410
Medio Bajo    0.249239
Alto          0.247599
Name: proportion, dtype: float64

Probabilidades Marginales de loan_status:
 loan_status
 Approved    0.62216
 Rejected    0.37784
Name: proportion, dtype: float64


In [198]:
from functools import reduce

import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hipotesis = None


    def __init__(self,filename=None, class_attr=None ):
        self.data = pd.read_csv(filename, sep=',', header =(0))
        self.class_attr = class_attr

    '''
        probabilidad(clase) =   frecuencia de aparcicin or columna/ conteo clase
    '''
    def calcular_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data =  list(self.data[self.class_attr])
        for i in class_values:
            self.priori[i]  = class_data.count(i)/float(len(class_data))
        print ("Valores a priori : ", self.priori)

    '''
        Calculo de las probabilidades individuales
        P(outcome|evidencia) =   P(verosimilitud de la evidencia) x Prior prob de outcome/P(Evidencia)

    '''
    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total =1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total+=1
        return total/float(class_data.count(class_value))

    '''
        Aquí se calcula la Probabilidad de Evidencia y se multiplica todas las probabilidades individuales a priori
        (Outcome|Multiple Evidence) = P(Evidence1|Outcome) x P(Evidence2|outcome) x ... x P(EvidenceN|outcome) x P(Outcome)
        escalado por P(Multiple Evidence)
    '''
    def calcular_probabilidad_condicional(self, hipotesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hipotesis:
                self.cp[i].update({ hipotesis[j]: self.get_cp(j, hipotesis[j], i)})
        print ("\nProbabilidades condicionales calculadas: \n")
        pprint.pprint(self.cp)

    def clasificar(self):
        print ("Resultado: ")
        for i in self.cp:
            print (i, " ==> ", reduce(lambda x, y: x*y, self.cp[i].values())*self.priori[i])

if __name__ == "__main__":
    c = Classifier(filename="loan.csv", class_attr=" loan_status" )
    c.calcular_priori()
    c.hipotesis = {" luxury_assets_value":'Alto'}

    c.calcular_probabilidad_condicional(c.hipotesis)
    c.clasificar()

Valores a priori :  {' Rejected': 0.3778402436167721, ' Approved': 0.622159756383228}

Probabilidades condicionales calculadas: 

{' Approved': {'Alto': 0.00037650602409638556},
 ' Rejected': {'Alto': 0.0006199628022318661}}
Resultado: 
 Rejected  ==>  0.00023424689622862497
 Approved  ==>  0.000234246896228625


In [228]:
from functools import reduce
import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hypothesis = None

    def __init__(self, filename=None, class_attr=None):
        self.data = pd.read_csv(filename, sep=',', header=(0))
        self.class_attr = class_attr

    def calculate_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data = list(self.data[self.class_attr])
        total_instances = float(len(class_data))

        for i in class_values:
            self.priori[i] = class_data.count(i) / total_instances
        print("Priori Values: ", self.priori)

    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total = 1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total += 1
        #print(class_value)
        #print(total, ' / ' , class_data.count(class_value) ,' + ', len(set(data_attr)))
        return total / float(class_data.count(class_value) + len(set(data_attr)))
    def calculate_conditional_probabilities(self, hypothesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hypothesis:
                #print(j)
                self.cp[i].update({hypothesis[j]: self.get_cp(j, hypothesis[j], i)})
        print("\nCalculated Conditional Probabilities: \n")
        pprint.pprint(self.cp)

    def classify(self):
        print("Result: ")
        total_prob = 0  
        for i in self.cp:
            total_prob += reduce(lambda x, y: x * y, self.cp[i].values()) * self.priori[i]

        # Imprimir los resultados normalizados
        for i in self.cp:
            prob = (reduce(lambda x, y: x * y, self.cp[i].values()) * self.priori[i]) / total_prob
            print(i, " ==> ", prob)


if __name__ == "__main__":
    c = Classifier(filename="loan.csv", class_attr=" loan_status")
    c.calculate_priori()
    c.hypothesis = {" luxury_assets_value":'Medio Bajo', " education": ' No Graduate', ' self_employed': ' No'}

    c.calculate_conditional_probabilities(c.hypothesis)
    c.classify()

Priori Values:  {' Rejected': 0.3778402436167721, ' Approved': 0.622159756383228}

Calculated Conditional Probabilities: 

{' Approved': {' No': 0.4962377727614748,
               ' No Graduate': 0.0003762227238525207,
               'Medio Bajo': 0.00032948929159802305},
 ' Rejected': {' No': 0.496594427244582,
               ' No Graduate': 0.0006191950464396285,
               'Medio Bajo': 0.000502008032128514}}
Result: 
 Rejected  ==>  0.6037953141004211
 Approved  ==>  0.3962046858995789
